## Explore : Multi Agent-Systems in Auto-gen

 A team is a group of agents that work together to achieve a common goal

In [1]:
# Necessary imports and setup
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GEMINI_API_KEY')
model_client = OpenAIChatCompletionClient(model = 'gemini-2.0-flash',
                                          api_key = api_key)

## 1. Single Agent Approach

A single agent is going to create a short story about a given topic

In [2]:
from autogen_agentchat.agents import AssistantAgent
# define a story agent
story_agent = AssistantAgent(
    name = 'story_agent',
    model_client = model_client,
    system_message = 'You are a creating writer. Generate a short story about the given topic.'
)

In [3]:
from autogen_agentchat.messages import TextMessage
# create a task for the agent
async def test_simple_agent():
    task = TextMessage(
        content = 'Write a short story about a college introvert boy as main superhero character.' \
                  'Keep it up to 50 words.',
        source = 'user'
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)

await test_simple_agent()

Ethan, a college introvert, found solace in books and coding. One night, a meteor shower bestowed him with telekinetic powers. He became "Page Turner," anonymously rescuing people, always retreating to his quiet dorm, a hero hidden in plain sight, his secret safe within the pages of his books.



Ethan, a college introvert, found solace in books and coding. One night, a meteor shower bestowed him with telekinetic powers. He became "Page Turner," anonymously rescuing people, always retreating to his quiet dorm, a hero hidden in plain sight, his secret safe within the pages of his books.


## 2. Multi Agent Team Approach

let's uses multiple agents to now write the story, each with its own work and mastery

In [4]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client = model_client,
    system_message = "You create engaging plots for stories."
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client = model_client,
    system_message = "You develop characters. Create interesting main characters with depth and background."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client = model_client,
    system_message = "You wrute engaging endings. conclude the story with a twist."
)


RoundRobinGroupChat is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat
# create a team with the agents
team = RoundRobinGroupChat(
    participants= [plot_agent, character_agent, ending_agent],
    max_turns = 6
)
team.component_type

'team'

In [6]:
async def test_team():
    task = TextMessage(
        content = 'Write a short story a brave boy and his Pokemon. Keep it up to 50 words',
        source = 'user'
    )

    result = await team.run(task = task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
        
await test_team()

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words
plot_writer : Toby, small but brave, clutched his partner, a fiery Charmander named Cinder. A shadowy figure loomed, stealing Pokemon. Toby, despite his fear, stepped forward. "Give them back!" Cinder roared, flames erupting. The thief faltered. Toby, with Cinder's help, rescued the stolen Pokemon, a true hero.

character_writer : Okay, I can do that. Here's a short story about a brave boy and his Pokemon:

Finnigan, freckled and timid, adored his timid Poliwag, Wiggles. A bully cornered them, demanding Wiggles. Finnigan, heart pounding, refused. "He's my friend!" Wiggles, sensing Finnigan's resolve, unleashed a surprisingly strong Water Gun. The bully fled, and Finnigan, emboldened, hugged Wiggles tight.

ending_writer : Okay, I can do that. Here's a short story about a brave boy and his Pokemon:

Toby, small but brave, clutched his partner, a fiery Charmander named Cinder. A shadowy figure loomed, stealing

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words  

--------------------------------------------------------------------------------------------------------------------  
plot_writer : Toby, small but brave, clutched his partner, a fiery Charmander named Cinder. A shadowy figure loomed, stealing Pokemon. Toby, despite his fear, stepped forward. "Give them back!" Cinder roared, flames erupting. The thief faltered. Toby, with Cinder's help, rescued the stolen Pokemon, a true hero.

character_writer : Okay, I can do that. Here's a short story about a brave boy and his Pokemon:

Finnigan, freckled and timid, adored his timid Poliwag, Wiggles. A bully cornered them, demanding Wiggles. Finnigan, heart pounding, refused. "He's my friend!" Wiggles, sensing Finnigan's resolve, unleashed a surprisingly strong Water Gun. The bully fled, and Finnigan, emboldened, hugged Wiggles tight.

ending_writer : Okay, I can do that. Here's a short story about a brave boy and his Pokemon:

Toby, small but brave, clutched his partner, a fiery Charmander named Cinder. A shadowy figure loomed, stealing Pokemon. Toby, despite his fear, stepped forward. "Give them back!" Cinder roared, flames erupting. The thief faltered. Toby, with Cinder's help, rescued the stolen Pokemon, a true hero. But the shadowy figure merely chuckled, removing their mask, revealing... Toby's future self, warning him of a darker path ahead.  

--------------------------------------------------------------------------------------------------------------------  
plot_writer : That's a great twist! Here's another take, playing on that darker path theme, also under 50 words:

Toby, once brave, now colder, stood with his Charizard, Cinder. He commanded Cinder to attack a defenseless trainer, stealing their Pokemon. A small voice cried out, "Stop!" Toby turned, seeing a younger version of himself, hand outstretched to a timid Charmander. Regret flickered. Was he already too far gone?

character_writer : That's powerful! Here's one building on *your* darker path theme, keeping it under 50 words:

Toby, his face scarred, and his Charizard, Cinder, stood victorious over fallen trainers. "Power," Toby rasped, clutching a stolen badge. Cinder nudged him, a flicker of warmth in his eyes. Toby shoved him away. "Don't you understand? We *need* this." A tear traced a path through the grime on his cheek.

ending_writer : That's fantastic! Okay, here's one continuing *your* version, under 50 words:

Toby, adorned with stolen badges, faced his reflection. A ghost of the brave boy remained, horrified. Cinder nuzzled him, whimpering. Toby saw the fear in his partner's eyes. He ripped off the badges, dropping them. "I... I can't," he choked, collapsing, Cinder's warmth the only comfort in the gathering dark.


### Comparision Table [Single V/S Multi Agents]

#### Single-Agent vs. Multi-Agent Comparison

| Aspect             | Single Agent                | Multi-Agent Team              |
|--------------------|-----------------------------|-------------------------------|
| **Creativity**     | Limited to one viewpoint    | Diverse ideas from each agent |
| **Depth**          | Basic story elements        | Detailed plot, characters, ending |
| **Flexibility**    | Stuck to one style          | Adaptable with specialized roles |
| **Setup Effort**   | Simple, one agent           | More agents, team setup       |

## Using Task Result

In Autogen AgentChat, tasks are usually units of work you give to an AI agent (like answer this question, run this code, fetch data).  
  
1. When the task finishes, the framework wraps the outcome in a TaskResult object.  
  
2. That object standardizes what the result looks like, instead of just returning a raw string or dict.  
  
While exact fields depend on the version of autogen, a TaskResult usually includes:  
  
content → the actual answer or output of the task (text, JSON, etc.).  
  
success (or similar flag) → whether the task completed successfully.  
  
metadata → extra info like execution time, cost, or trace details.  

In [7]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task = "Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='ab06bfed-5315-49d1-918e-033bd2a42b51' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 31, 12, 41, 5, 910266, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='382f1784-8876-468f-8e2c-b60222536458' source='plot_writer' models_usage=RequestUsage(prompt_tokens=16, completion_tokens=74) metadata={} created_at=datetime.datetime(2025, 8, 31, 12, 41, 8, 513511, tzinfo=datetime.timezone.utc) content="The air grows crisp, a whispered chill,\nLeaves dance and twirl upon the hill.\nGold, crimson, amber, brown,\nA fiery carpet drifting down.\n\nThe sun hangs low, a gentle grace,\nAs summer yields to autumn's space.\nA time for harvest, stories told,\nBefore the winter's icy hold.\n" type='TextMessage'
id='f1b29d14-a9d9-4f15-9be0-7fa4f5b32666' source='character_writer' models_usage=RequestUsage(prompt_tokens=96, completion_tokens=442) metadata={} created_at=datetime.datetime(2025, 8, 31, 12, 41, 11, 849142,

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat

team2 = RoundRobinGroupChat(
    participants= [plot_agent],
    max_turns = 2
)

In [9]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team2.reset()  # Reset the team for a new task.
async for message in team2.run_stream(task = "Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='a8422f78-65c0-47ee-9c85-c39e01c43d6c' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 31, 12, 46, 41, 495616, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='09879a5d-b73c-43ab-b911-36d0295c78b1' source='plot_writer' models_usage=RequestUsage(prompt_tokens=16, completion_tokens=92) metadata={} created_at=datetime.datetime(2025, 8, 31, 12, 46, 42, 754213, tzinfo=datetime.timezone.utc) content="Okay, here's a short poem about the fall season:\n\nThe air grows crisp, a golden haze,\nWhere summer's green now softly sways.\nLeaves dance and twirl, a fiery show,\nIn hues of red and amber glow.\n\nThe wind whispers secrets through the trees,\nA gentle promise of winter's ease.\nPumpkins gleam, a harvest bright,\nAs days grow short and stars ignite.\n" type='TextMessage'
id='a1994fb1-34ae-4910-b91b-db977167fbab' source='plot_writer' models_usage=RequestUsage(prompt_tokens=108, completion_tokens=0) me

In [10]:
message

TaskResult(messages=[TextMessage(id='a8422f78-65c0-47ee-9c85-c39e01c43d6c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 31, 12, 46, 41, 495616, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='09879a5d-b73c-43ab-b911-36d0295c78b1', source='plot_writer', models_usage=RequestUsage(prompt_tokens=16, completion_tokens=92), metadata={}, created_at=datetime.datetime(2025, 8, 31, 12, 46, 42, 754213, tzinfo=datetime.timezone.utc), content="Okay, here's a short poem about the fall season:\n\nThe air grows crisp, a golden haze,\nWhere summer's green now softly sways.\nLeaves dance and twirl, a fiery show,\nIn hues of red and amber glow.\n\nThe wind whispers secrets through the trees,\nA gentle promise of winter's ease.\nPumpkins gleam, a harvest bright,\nAs days grow short and stars ignite.\n", type='TextMessage'), TextMessage(id='a1994fb1-34ae-4910-b91b-db977167fbab', source='plot_wr

In [15]:
from autogen_agentchat.teams import RoundRobinGroupChat

team3 = RoundRobinGroupChat(
    participants= [plot_agent],
    # max_turns=2
)

In [ ]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team3.reset()  # Reset the team for a new task.
async for message in team3.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 9, 18, 365921, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=103) metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 9, 21, 780708, tzinfo=datetime.timezone.utc) content="Leaves of amber gently fall,  \nDancing to the autumn's call.  \nWhispers of the cooling breeze,  \nWeaving through the golden trees.\n\nPumpkins gleam in sunset hues,  \nBeneath a sky of deeper blues.  \nSweaters warm against the chill,  \nAs twilight wraps the quiet hill.\n\nCrisp and sweet, the air does beam,  \nPainting worlds that softly dream.  \nIn fall's embrace, nature's art,  \nA season's song within the heart.  " type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=141, completion_tokens=1) metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 9,